In [ ]:
! pip install accelerate peft bitsandbytes pip install git+https://github.com/huggingface/transformers trl py7zr auto-gptq optimum

In [ ]:
import torch
from datasets import load_dataset, Dataset
from peft import LoraConfig, AutoPeftModelForCausalLM, prepare_model_for_kbit_training, get_peft_model
from transformers import AutoModelForCausalLM, AutoTokenizer, GPTQConfig, TrainingArguments
from trl import SFTTrainer
import os
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
path = "/content/drive/MyDrive/Inlab/Datasets/Prova_Ner/"
path = "/content/drive/MyDrive/MAI/Q3/HLE/Project/Data/Finetuning/Mistral/First Format/"
train_data = pd.read_csv(path+"train.csv")
dev_data = pd.read_csv(path+"dev.csv")
test_data = pd.read_csv(path+"test.csv")

In [ ]:
train_dataset = Dataset.from_pandas(train_data)
dev_dataset = Dataset.from_pandas(dev_data)
test_dataset = Dataset.from_pandas(test_data)

In [ ]:
test_data['text'].iloc[0]

In [ ]:
# model_name = "TheBloke/Mistral-7B-Instruct-v0.1-AWQ"
model_name = "TheBloke/Mistral-7B-Instruct-v0.1-GPTQ"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")
print(model)

In [ ]:
# def generate_and_tokenize_prompt(prompt):
#     return tokenizer(prompt)

# tokenized_train_dataset = train_dataset.map(generate_and_tokenize_prompt)
# tokenized_val_dataset = dev_dataset.map(generate_and_tokenize_prompt)
tokenizer.padding_side = "right"

In [ ]:
model.config.use_cache=False
model.config.pretraining_tp=1
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)
peft_config = LoraConfig(
    r=16,
    lora_alpha=64,
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj",
                    # "up_proj",
                    # "o_proj",
                    # "k_proj",
                    # "down_proj",
                    # "gate_proj",
                    "v_proj"]
)
model = get_peft_model(model, peft_config)
training_arguments = TrainingArguments(
    output_dir="/content/drive/MyDrive/MAI/Q3/HLE/Project/Models/Mistral_finetuned_ner/", #Important indicar be la direcció
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    do_eval=True,
    evaluation_strategy="steps",
    eval_steps=50,
    max_grad_norm=0.3,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    learning_rate=2e-4,
    lr_scheduler_type="constant",
    save_strategy="steps",
    save_steps=50,
    logging_strategy='steps',
    logging_steps = 50,
    num_train_epochs=1,
    # max_steps=250,
    group_by_length = True,
    fp16=True,
    push_to_hub=False
)
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=dev_dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    args=training_arguments,
    tokenizer=tokenizer,
    packing=False,
    max_seq_length=1024 #mirar si es adecuat
)

In [ ]:
trainer.train()

In [ ]:
# ! cp -r /content/mistral-finetuned-ner/checkpoint-200/ /content/drive/MyDrive/MAI/Q3/HLE/Project/Models/Mistral_7B_200/

In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

# Inference

In [ ]:
test_data['Sentence'].iloc[0]

In [ ]:
instruction = "You are solving the NER problem. Extract from the text words related to each of the following entities: CASE_NUMBER, COURT, DATE, GPE, JUDGE, LAWYER, ORG, OTHER_PERSON, PETITIONER, PRECEDENT, PROVISION, RESPONDENT, STATUTE, WITNESS."
sentence = test_data['Sentence'].iloc[0]
text = f"""<s>[INST] {instruction} Here is the text: {sentence} [/INST]\n"""

In [ ]:
from peft import AutoPeftModelForCausalLM
from transformers import GenerationConfig
from transformers import AutoTokenizer
import torch
# tokenizer = AutoTokenizer.from_pretrained("/content/mistral-finetuned-tickets/checkpoint-13")


# model = AutoPeftModelForCausalLM.from_pretrained(
#     "/content/mistral-finetuned-tickets/checkpoint-60",
#     low_cpu_mem_usage=True,
#     return_dict=True,
#     torch_dtype=torch.float16,
#     device_map="cuda")


In [ ]:
generation_config = GenerationConfig(
    do_sample=True,
    top_k=1,
    temperature=0.1,
    max_new_tokens=500,
    pad_token_id=tokenizer.eos_token_id
)

In [ ]:
import time
st_time = time.time()
inputs = tokenizer(text, return_tensors="pt").to("cuda")
outputs = model.generate(**inputs, generation_config=generation_config)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))
print(time.time()-st_time)

In [ ]:
from huggingface_hub import notebook_login

notebook_login() #### Important donar token de write
#create repo of the model

In [ ]:
from huggingface_hub import Repository

# Create a local directory to save your model
!mkdir Mistral-7B-LegalNER

repo = Repository(local_dir="Mistral-7B-LegalNER", clone_from="Huss9/Mistral-7B-LegalNER")
repo.git_pull()
!cp -r /content/drive/MyDrive/MAI/Q3/HLE/Project/Models/Mistral_finetuned_ner/checkpoint-250/* ./Mistral-7B-LegalNER/
repo.git_add(auto_lfs_track=True)
repo.git_commit("After 250 steps")
repo.git_push()

In [ ]:
repo.git_add(all=True)